**READ ME**
**Purpose**: This script makes a list of all FSAs in the provinces of interest (in this case, MB, AB, and SK), pulls demographic information from the StatCan page of each FSA, and returns the demographic information (along with information on whether the FSA corresponds to a rural (<1,000), small (1,000-30,000), medium (30,000-1,000,000) or large (>1,000,000) urban center). The goal of this project was to ensure that our analysts collected data that is representative of the rural/urban populations of each of these provinces. 

1. Functions used in this script:
    - **find_tables(url)**: Takes wikipedia url as argument and returns all tables on the page
    - **get_table(url)**: Takes a wikipedia url for population centres of a province as argument and returns a df with the cities and the city size
    - **urban_fsa(table)**:Takes a df as an argument and performs prelimiary processing on that table to get a table with FSAs and corresponding city/region
    - **rural_fsa(table)**:Takes a df as an argument and performs prelimiary processing on that table to get a table with FSAs and corresponding city/region
    - **city_size_info(table)**: Takes a table as argument and returns a list of cities that are missing information on size of the city
    - **remove_spaces(cities)**: Takes list of cities as argument and replaces spaces with underscores, which is needed to general Wikipedia URLs for those cities
    - **wiki_url(city)**: Takes a city as argument and returns a URL of the form 'https://en.wikipedia.org/wiki/[city]'
    - **province_url(city, province)**: Takes a city and province as arguments and returns a URL of the form 'https://en.wikipedia.org/wiki/[city],_[province]'
    - **municipality_url(city)**: Takes a city as argument and returns a URL of the form 'https://en.wikipedia.org/wiki/Rural_Municipality_of_[city]'
    - **find_infobox_table(url)**: Takes wikipedia page as argument and returns the infobox table on that page
    - **get_pop_info(cities, province)**: Takes a list of cities and the province they are in as arguments and returns wikipedia URLs for each of those cities so that we can subsequently scrape the infobox for that city
    - **pop_number(url, city)**: Takes a URL and a city and finds the population data associated with that city
    - **def pop_data(urls)**: This takes a list of StatCan URLs as argument and pulls desired demographic information from each URL, stores in a dictionary, and returns a dataframe

Steps to the script:
1. Get a list of cities in each province from the Wikipedia page for that [province](https://en.wikipedia.org/wiki/List_of_population_centres_in_Alberta) and return a df of cities and the city size for the province.
2. Create a df of all the FSAs in the province from the Wikipedia page of FSAs for that [province](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T) (this df should include both rural and urban FSAs, which may mean scraping 2 df for some of the provinces).
3. Concat df of cities in the province with df of FSAs to obtain a df that has all the FSAs in the province, the city/region, and the city/region size. 
4. Create a list of all the cities/regions missing city/region size information (because not all the cities listed in the population centres page appear in the postal codes page) and scrape the wikipedia page for each [city](https://en.wikipedia.org/wiki/Cardston,_Alberta) to find the population size. Then, sort into small, med, large urban, or rural environment. 
5. Create list of all FSAs that we want to obtain data for on [StatCan](https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=FSA&Code1=T1A&Geo2=PR&Code2=01&SearchText=T1A&SearchType=Begins&SearchPR=01&B1=All&TABID=2&type=0). Scrape StatCan pages and store information in a df. 
6. Concat this df with the df of city/region and city/region size df and export to csv. 

In [1]:
#import packages
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup

In [2]:
#first, check that we can scrape the Statscan page.
response = requests.get(
	url="https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=FSA&Code1=S0A&Geo2=PR&Code2=01&SearchText=S0A&SearchType=Begins&SearchPR=01&B1=All&TABID=2&type=0",
)
print(response.status_code)

#200 means we are allowed to scrape the page
#wikipedia also returns 200

200


In [3]:
#Get a list of cities in each province from the Wikipedia page for that province and return a df of cities and the city size for the province.

#This function takes a 'List of population centres in [province]' Wikipedia URL and returns a df of all city centres and size (sml, med, lrg)
def get_table(url):
    # parse data from the html into a beautifulsoup object
    response=requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table=soup.find('table',{'class':"wikitable"})
    df=pd.read_html(str(table))
    df=pd.DataFrame(df[0]) # convert list to dataframe
    df.columns.values[1]='City'
    df.columns.values[2]='City size'
    df = df[['City','City size']]
    return df

#Define URLs we want to parse 
alberta_cities = 'https://en.wikipedia.org/wiki/List_of_population_centres_in_Alberta'
manitoba_cities = 'https://en.wikipedia.org/wiki/List_of_population_centres_in_Manitoba'
saskatchewan_cities = 'https://en.wikipedia.org/wiki/List_of_population_centres_in_Saskatchewan'

#Get table of cities and population size of cities for each Province
alberta_cities = get_table(alberta_cities)
manitoba_cities = get_table(manitoba_cities)
saskatchewan_cities = get_table(saskatchewan_cities)

In [4]:
#Create a df of all the FSAs in the province from the Wikipedia page of FSAs for that province 
#(this df should include both rural and urban FSAs, which may mean scraping 2 df for some of the provinces).

#Define a function that takes a 'Lits of postal codes of [province]' Wikipedia URL and returns all the tables on that page
def find_tables(url):
    response=requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tables=soup.findAll('table')
    html = requests.get(url).content
    all_df = pd.read_html(html)
    return all_df

#Define postal code URLS to parse
ab_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T'
mb_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_R'
sk_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_S'

#Define tables for AB
ab_tables = find_tables(ab_url) #ab_tables corresponds to all the tables on the wikipedia page
ab_urban_table = ab_tables[1] #create a df with all urban FSAs
ab_rural_table = ab_tables[2] #create a df with all rural FSAs

#Define tables for MB
mb_tables = find_tables(mb_url) 
mb_urban_table = mb_tables[0] 
mb_rural_table = mb_tables[1] 

#Define tables for SK
sk_tables = find_tables(sk_url) 
sk_urban_table = sk_tables[0] 
sk_rural_table = sk_tables[1] 

In [5]:
#Perform pre-processing of urban FSA tables

#For AB
ab_urban_table_copy = ab_urban_table
ab_urban_table_copy.drop(columns=['Neighbourhood','Latitude','Longitude'], inplace=True)
ab_urban_table_copy.rename(columns={'Postal Code':'FSA', 'Community':'City/Region'}, inplace=True)
ab_urban_table_copy['City/Region'] = ab_urban_table_copy['City/Region'].str.split(',').str[0]
ab_urban_table_copy = ab_urban_table_copy[ab_urban_table_copy['City/Region'].str.contains('Not assigned')==False]
pd.set_option('display.max_rows', 500)
ab_urban_table_copy

#join the urban table with the list of cities so we get correspondence of FSA and city size
ab_urban_table_copy = pd.merge(ab_urban_table_copy, alberta_cities, how='left', left_on='City/Region', right_on='City')
ab_urban_table_copy.drop(columns=['City'], inplace = True)

#This function performs preliminary processing on the urban FSA table scraped from wikipedia. 
#We couldn't use this function for AB because the formatting is different for AB than it is for MB and SK
def urban_fsa(table):
    table = pd.melt(table, value_name='INFO')
    table.drop('variable', inplace=True, axis=1)
    table = table[~table['INFO'].str.contains("Not assigned")]
    table['FSA'] = table['INFO'].str[:3] #Create an FSA column with first 3 characters of the Info column
    table['City/Region'] = table['INFO'].str[3:] #Create a region column with the region information 
    table['City/Region'] = table['City/Region'].str.split('(').str[0]
    table['City/Region'] = table['City/Region'].str.split('/').str[0]
    table.drop('INFO', inplace=True, axis=1)#Delete info table
    return table

#For MB
mb_urban_table_copy = urban_fsa(mb_urban_table) 
mb_urban_table_copy['City/Region'] = mb_urban_table_copy['City/Region'].replace({'HeadingleyEast' : 'Headingley', #Rename cities that need to be renamed
                                                               'HeadingleyWest' : 'Headingley',
                                                               'BrandonSouth':'Brandon',
                                                               'BrandonEast and vicinity':'Brandon',
                                                               'BrandonNorthwest':'Brandon'})

mb_urban_table_copy = pd.merge(mb_urban_table_copy, manitoba_cities, how='left', left_on='City/Region', right_on='City')
mb_urban_table_copy.drop(columns=['City'], inplace = True)

#For SK
sk_urban_table_copy = urban_fsa(sk_urban_table)
sk_urban_table_copy['City/Region'].replace({'North Battleford' : 'Battleford'}, inplace=True)
delimiters = ['North', 'South', 'East', 'West', 'Central', 'Outer', 'Various', 'and'] #Remove all substrings in the Region column of the df that don't correspond to a city.
for delimiter in delimiters:
    sk_urban_table_copy['City/Region'] = sk_urban_table_copy['City/Region'].str.split(delimiter).str[0]
sk_urban_table_copy['City/Region'].replace({'Battleford' : 'North Battleford'}, inplace=True)
sk_urban_table_copy['City/Region'] = sk_urban_table_copy['City/Region'].str.rstrip() #Remove whitespace on right

sk_urban_table_copy = pd.merge(sk_urban_table_copy, saskatchewan_cities, how='left', left_on='City/Region', right_on='City')
sk_urban_table_copy.drop(columns=['City'], inplace = True)

In [6]:
#pre-process rural fsa df 

#write a function that performs preliminary processing on the rural FSA table scraped from wikipedia
def rural_fsa(table):
    table = pd.melt(table, value_name='INFO')
    table.drop('variable', inplace=True, axis=1)
    table = table[~table['INFO'].str.contains("Not in use")]
    table = table[~table['INFO'].str.contains("Commercial")]
    delimiters = ['0A0','0A1','1A0','1B0','Commercial']
    for delimiter in delimiters:
        table['INFO'] = table['INFO'].str.split(delimiter).str[0] #Remove everything after the FSA and the region in the FSA column (we are removing postal code information for rural cities that is not needed at the moment)
    table['INFO'] = table['INFO'].str.split('0A1').str[0]
    table['FSA'] = table['INFO'].str[:3] #Create an FSA column with first 3 characters of the Info column
    table['City/Region'] = table['INFO'].str[3:] #Create a region column with the region information 
    table['City size'] = 'Rural'
    table.drop('INFO', inplace=True, axis=1)
    return table

#For all provinces
ab_rural_table_copy = rural_fsa(ab_rural_table) #nothing further needs to be done here in terms of pre-processing
mb_rural_table_copy = rural_fsa(mb_rural_table)
sk_rural_table_copy = rural_fsa(sk_rural_table)

In [7]:
#Concat urban and rural tables for each provinces to return tables of all FSAs in the province
ab_table = pd.concat([ab_urban_table_copy, ab_rural_table_copy], axis = 0)
mb_table = pd.concat([mb_urban_table_copy, mb_rural_table_copy], axis = 0)
sk_table = pd.concat([sk_urban_table_copy, sk_rural_table_copy], axis = 0)

In [8]:
#The tables above have cities/regions that do not have city size. We want to retrieve this information using the wikipedia page for each city.

#This funcion takes a table and returns a list of cities that do not have city size data.  
def city_size_info(table):
    missing_mask = table['City size'].isna() #Creates Boolean mask of cities missing city size information
    missing_df = table[missing_mask] #Creates dataframe of cities missing size information using Boolean mask
    missing_list = missing_df['City/Region'].tolist() #Create a list from the column of all the cities that are missing information
    final_list=[]
    for item in missing_list:
        item = item.strip()
        final_list.append(item)
        #Need something here to delete duplicate items
    remove_duplicates = []
    for item in final_list: #removes duplicates from the list
        if item not in remove_duplicates:
            remove_duplicates.append(item)
    return remove_duplicates

missing_data_ab = city_size_info(ab_table) #this function returns a list of cities that have NaN as city size
missing_data_mb = city_size_info(mb_table)
missing_data_sk = city_size_info(sk_table)

In [9]:
#Now that we have lists of cities missing data, we use the following function to generate urls for each city that is missing information

def add_underscore(city):
    city = city.replace(' ', '_')
    return city

def get_missing_data_urls(list_cities, province):
    city_dict={}
    base_url = 'https://en.wikipedia.org/wiki/[city],_[province]'
    new_list = []
    for city in list_cities:
        city_underscore = add_underscore(city)
        url = base_url.replace('[city]', city_underscore)
        url = url.replace('[province]', province)
        city_dict.update({city: url})
    return city_dict

ab_missing_urls = get_missing_data_urls(missing_data_ab, 'Alberta')
mb_missing_urls = get_missing_data_urls(missing_data_mb, 'Manitoba')
sk_missing_urls = get_missing_data_urls(missing_data_sk, 'Saskatchewan')

#We have generated the URLs, and now we need to check whether or not they are valid pages on Wikipedia. 
#All valid pages are appended to a dictionary to be scraped

#Define a function that takes a URL and returns the infobox on a wikipedia page
def find_infobox_table(url):
    response=requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table=soup.find('table',{'class':"infobox"})
    return table

def get_pop_info(dict_cities):
    error_city_urls = []
    valid_url_dict = {}
    for key, value in dict_cities.items():
        table = find_infobox_table(value) #Retrieve the infobox table on the wikipedia page 
        if table == None: #If we can't find an infobox table, this means that we have landed on a disambiguation wikipedia page. 
            error_city_urls.append(key)
        else:
            valid_url_dict.update({key: value})
    print('URLS for the following cities could not be found:')
    print(error_city_urls)
    return valid_url_dict

ab_valid_urls = get_pop_info(ab_missing_urls)
mb_valid_urls = get_pop_info(mb_missing_urls)
sk_valid_urls = get_pop_info(sk_missing_urls)

URLS for the following cities could not be found:
["Tsuut'ina"]
URLS for the following cities could not be found:
['Narol', 'Ste-Geneviève', 'Sunnyside', 'Navin']
URLS for the following cities could not be found:
[]


In [10]:
#Write a function that allows you to find the population data on a Wikipedia page:

def pop_number(cities_dict):  
    pop_size = {}
    list = []
    for key, value in cities_dict.items():      
        city_name = key #Get the name of the city we are getting population data for
        pop_size.update({'City/Region': city_name}) #Append this name to the pop_size dictionary
            
        table = find_infobox_table(value)
        df=pd.read_html(str(table))
        df=pd.DataFrame(df[0])
        
        #Use this df to extract the population information we need
        col1 = df.iloc[: , 0].str.split('(').str[0] #Select the first column of the df as a series and remove extraneous information in parentheses
        col1 = col1.str.strip() #Strip all whitespace
        df=df.set_index(col1) #Set first column as index
        if 'Population' in df.index:
            pop_idx = df.index.tolist().index('Population') #Find the index of the population row
            pop = df.iat[pop_idx+1, 1] #Define the index (row and column) of the population number, which is located right below the population row
            pop = pop.partition(' ')[0] #Only keep first item in string, which is the population number
            if ',' in pop:
                pop = pop.replace(',', '') #Remove comma from number if there is one
            pop = pop.split('[')[0]
            pop = int(pop) #Convert string to integer
        
        else:
            pop = 0

        #Classify the size of the city based on the pop integer
        if 0 <= pop < 1000:
            city_size = 'Rural'
        elif 1000 <= pop <30000:
            city_size = 'Small'
        elif 30000 <= pop < 1000000:
            city_size = 'Medium'
        else: 
            city_size = 'Large urban'
            
        pop_size.update({'City size': city_size}) #Update the dictionary of the City/Region and population size
        list.append(pop_size.copy())
            
    df = pd.DataFrame(list)
    return df

ab_missing_pop = pop_number(ab_valid_urls)
mb_missing_pop = pop_number(mb_valid_urls)
sk_missing_pop = pop_number(sk_valid_urls)

In [11]:
#This function takes the original table and the table with the data for cities missing a city size, and merges them
def get_full_fsa_table(original_table, missing_data_table):
    final_table = pd.merge(original_table, missing_data_table, how='left', on='City/Region')
    final_table['City size'] = final_table['City size_x'].astype(str)+final_table['City size_y'].astype(str)
    final_table['City size'] = final_table['City size'].replace('nan','', regex=True)
    final_table['City size'] = final_table['City size'].replace('','Rural', regex=True) #Replace all other values with NaN
    final_table.drop(columns=['City size_x', 'City size_y'], inplace=True)
    return final_table

ab = get_full_fsa_table(ab_table, ab_missing_pop)
mb = get_full_fsa_table(mb_table, mb_missing_pop)
sk = get_full_fsa_table(sk_table, sk_missing_pop)

In [12]:
#This function takes a table as argument and converts the FSA column to a list, then returns a list of StatCan URLs to parse
def urls_list(table): 
    base_url = 'https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=FSA&Code1=[x]&Geo2=PR&Code2=01&SearchText=[x]&SearchType=Begins&SearchPR=01&B1=All&TABID=2&type=0'
    fsa_list = table['FSA'].tolist()
    new_urls=[]
    for item in fsa_list:
        new_url = base_url.replace('[x]', item)
        new_urls.append(new_url)
    return new_urls

ab_urls = urls_list(ab)
mb_urls = urls_list(mb)
sk_urls = urls_list(sk)

In [13]:
#This function iterates through every URL in a list of URLs, pulls desired demographic information, and stores it in a dictionary
def pop_data(urls):
    
    pop_dict={}
    full_dict_list=[]
    fsa_not_found=[]
    
    for url in urls:
        
        response=requests.get(url) 
        soup = BeautifulSoup(response.text, 'html.parser') #Read HTML on page
        table=soup.find('table',{'id':"tablehtml"}) #Find table on page
        df=pd.read_html(str(table)) #Read HTML table into a list of DF objects
        df=pd.DataFrame(df[0]) #Convert list to a df
        copy_df = df #Create a copy of the df
        
        copy_df["new_index"] = df["Characteristic"].astype(str) #Copy Characteristic column to use as an index
        sep = 'Census data footnote'
        copy_df["new_index"] = copy_df["new_index"].str.split('Census data footnote').str[0] #Remove all footnotes mentions in the index column
        copy_df = copy_df.set_index('new_index') #Set new_index as index
        
        fsa=url[102:105] #Define the FSA for that URL
        pop_dict.update({'FSA': fsa}) #Append the FSA in the dictionary
        
        if 'Population, 2016' in copy_df.index.tolist():
            pop_idx = copy_df.index.tolist().index('Population, 2016') #Find the row index for Population, 2016 and store in variable pop_idx
            pop = copy_df.iat[pop_idx, 1] #Find the pop number we are looking for
            pop_dict.update({'Total population': pop}) #Append to the dictionary
            
            #Repeat for all the data we want to scrape on the StatCan page
            indigenous_idx = copy_df.index.tolist().index('Aboriginal identity')
            indigenous = copy_df.iat[indigenous_idx, 1]
            pop_dict.update({'Indigenous total': indigenous})
    
            fn_idx = copy_df.index.tolist().index('First Nations (North American Indian)')
            fn = copy_df.iat[fn_idx, 1]
            pop_dict.update({'First Nations': fn})
    
            metis_idx = copy_df.index.tolist().index('Métis')
            metis = copy_df.iat[metis_idx, 1]
            pop_dict.update({'Métis': metis})
    
            inuk_idx = copy_df.index.tolist().index('Inuk (Inuit)')
            inuk = copy_df.iat[inuk_idx, 1]
            pop_dict.update({'Inuit': inuk})
    
            multiple_idx = copy_df.index.tolist().index('Multiple Aboriginal responses')
            multiple = copy_df.iat[inuk_idx, 1]
            pop_dict.update({'Multiple indigenous identities': multiple})
    
            other_idx = copy_df.index.tolist().index('Aboriginal responses not included elsewhere')
            other = copy_df.iat[other_idx, 1]
            pop_dict.update({'Other': other})
            
            full_dict_list.append(pop_dict.copy()) #Append every dictionary to a list
            
        else: #No data associated with the FSA
            pop_dict.update({'Total population': ''}) #Append to the dictionary
            pop_dict.update({'Indigenous total': ''})
            pop_dict.update({'First Nations': ''})
            pop_dict.update({'Métis': ''})
            pop_dict.update({'Inuit': ''})
            pop_dict.update({'Multiple indigenous identities': ''})
            pop_dict.update({'Other': ''})
            full_dict_list.append(pop_dict.copy()) #Append every dictionary to a list
            fsa_not_found.append(fsa)
    
    print('No StatCan data was found for the following FSAs:')
    print(fsa_not_found)
    df = pd.DataFrame(full_dict_list) #Convert the list 
    return df #Return the df

In [16]:
ab_df = pop_data(ab_urls)
mb_df = pop_data(mb_urls)
sk_df = pop_data(sk_urls)

No StatCan data was found for the following FSAs:
['T3T', 'T6Y', 'T0W']
No StatCan data was found for the following FSAs:
['R5J', 'R5K', 'R5L', 'R5M', 'R5N', 'R5P', 'R5R', 'R5T', 'R6P', 'R6T']
No StatCan data was found for the following FSAs:
['S4K', 'S7A', 'S7B', 'S7C']


In [19]:
#Merge the StatCan data df with the city size (rural, sml, med, lrg) df
ab_final = pd.merge(ab_df, ab, how='left', on='FSA', sort=True)
mb_final = pd.merge(mb_df, mb, how='left', on='FSA', sort=True)
sk_final = pd.merge(sk_df, sk, how='left', on='FSA', sort=True)

In [21]:
#Concat into one table and export to CSV
final_table = pd.concat([ab_final, mb_final, sk_final], axis = 0)
final_table['City size'].replace({'Large urban':'Large'}, inplace=True)
final_table.to_csv(r"C:\Users\macla\Desktop\statcan_output.csv", index=False)

In [22]:
final_table

,FSA,Total population,Indigenous total,First Nations,Métis,Inuit,Multiple indigenous identities,Other,City/Region,City size
0,T0A,59234,13095,6790,6125,70,70,20,,Rural
1,T0B,64072,3130,1125,1830,75,75,25,,Rural
2,T0C,62701,10510,8675,1720,75,75,35,,Rural
3,T0E,43729,5830,3520,2150,40,40,25,,Rural
4,T0G,42905,15755,10995,4560,10,10,65,,Rural
5,T0H,84336,17230,12235,4720,60,60,80,,Rural
6,T0J,50783,5220,4000,1130,40,40,20,,Rural
7,T0K,55487,3890,2800,1025,10,10,35,,Rural
8,T0L,61161,10810,9730,1020,30,30,15,,Rural
9,T0M,67484,3190,965,2130,20,20,20,,Rural
